#### Acknowledgement
This notebook for the creation of the annotated training data for the YOLO architecture is based on the repository at the following link \
https://github.com/geaxgx/playing-card-detection

### Imports

In [ ]:
import numpy as np
import cv2
import os
from tqdm import tqdm
import random
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import pickle
from glob import glob 
import imgaug as ia
from imgaug import augmenters as iaa
from shapely.geometry import Polygon

In [ ]:
def select_image(folder_dir, file_name):
    file_path = os.path.join(folder_dir, file_name)
    if os.path.exists(file_path):
        return cv2.imread(file_path)
    else:
        raise FileNotFoundError(f"Image {file_name} not found in {folder_dir}")

def select_random_image(folder_dir):
    file_names = os.listdir(folder_dir)
    random_file = random.choice(file_names)
    return select_image(folder_dir, random_file)

#### Card corner region calculation

In [ ]:
card_width = 57
card_height = 87
zoom_factor = 2.5

def extract_corners(filename, zoom=2.5):
    face_cards = ['J', 'Q', 'K']
    if any(face in filename for face in face_cards):
        x_min = 0.3
        x_max = 7.4
        y_min = 1
        y_max = 19
    else:
        x_min = 0.4
        x_max = 11
        y_min = 1
        y_max = 22
    x_min = int(x_min * zoom)
    x_max = int(x_max * zoom)
    y_min = int(y_min * zoom)
    y_max = int(y_max * zoom)
    return x_min, x_max, y_min, y_max

card_width = int(card_width * zoom_factor)
card_height = int(card_height * zoom_factor)

#### Utility functions

In [ ]:

def show_image(img,polygons = [],channels = "bgr",size=9):
    """
        Function to display an inline image, and draw optional polygons (bounding boxes, convex hulls) on it.
        Use the param 'channels' to specify the order of the channels ("bgr" for an image coming from OpenCV world)
    """
    if not isinstance(polygons,list):
        polygons = [polygons]    
    if channels == "bgr": # bgr (cv2 image)
        nb_channels = img.shape[2]
        if nb_channels == 4:
            img = cv2.cvtColor(img,cv2.COLOR_BGRA2RGBA)
        else:
            img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)    
    fig,ax = plt.subplots(figsize = (size,size))
    ax.set_facecolor((0,0,0))
    ax.imshow(img)
    for polygon in polygons:
        if len(polygon.shape) == 3:
            polygon=polygon.reshape(-1,2)
        patch = patches.Polygon(polygon,linewidth = 1,edgecolor = 'g',facecolor = 'none')
        ax.add_patch(patch)


def unique_filename(dirname, suffixes, index, prefix = ""):
    """
    Generate unique filenames in a specified directory with given suffixes.
    """

    if not isinstance(suffixes, list):
        suffixes=[suffixes]
    suffixes=[p if p[0] == '.' else '.'+p for p in suffixes]
    fnames = []
    for suffix in suffixes:
        # Save jpg and xml in separate folders
        if suffix == '.jpg':
            folder = os.path.join(dirname, 'images')
        elif suffix == '.xml':
            folder = os.path.join(dirname, 'annotations')
        else:
            folder = dirname
        if not os.path.exists(folder):
            os.makedirs(folder)
        fname = os.path.join(folder, f"{prefix}{index}{suffix}")
        fnames.append(fname)
    if len(fnames) == 1:
        return fnames[0]
    else:
        return fnames
    

def paste_card_clean(card_img, mask, background, x, y):
    """
    Paste card_img onto background at (x, y) using a blurred version of the mask.
    Automatically clips if card exceeds background size.
    """
    card_h, card_w = card_img.shape[:2]
    bg_h, bg_w = background.shape[:2]

    # Clip card and mask if it goes out of bounds
    paste_h = min(card_h, bg_h - y)
    paste_w = min(card_w, bg_w - x)
    
    if paste_h <= 0 or paste_w <= 0:
        print("Card goes out of bounds, skipping paste.")
        return background

    card_img = card_img[:paste_h, :paste_w]
    mask = mask[:paste_h, :paste_w]

    # Preprocess mask
    mask = cv2.resize(mask, (paste_w, paste_h))
    mask = mask.astype(np.float32) / 255.0
    mask = cv2.GaussianBlur(mask, (5, 5), 0)
    mask_3ch = np.stack([mask]*3, axis=-1)

    roi = background[y:y+paste_h, x:x+paste_w]
    blended = (card_img.astype(np.float32) * mask_3ch + roi.astype(np.float32) * (1 - mask_3ch)).astype(np.uint8)
    background[y:y+paste_h, x:x+paste_w] = blended
    return background




#### Global configuration

In [ ]:
card_img_dir = os.path.join(os.getcwd(), "data_classification/52_cards_mirrored")
suit_types = ['s', 'h', 'd', 'c']
value_types = ['A', 'K', 'Q', 'J', '10', '9', '8', '7', '6', '5', '4', '3', '2']
backgrounds_folder = os.path.join(os.getcwd(), "data_classification/backgrounds")
pickle_file = os.path.join(os.getcwd(), "utils_data/cards.pck")
imgW = 416
imgH = 416

#### Card region check

In [ ]:
card_img_files = glob(card_img_dir + "/*.png")
img_file = random.choice(card_img_files)
x_min, x_max, y_min, y_max = extract_corners(img_file.split("/")[-1])
ref_corner_1 = np.array([[x_min, y_min], [x_max, y_min], [x_max, y_max], [x_min, y_max]], dtype=np.float32)
ref_corner_2 = np.array([[card_width-x_max, card_height-y_max], [card_width-x_min, card_height-y_max], [card_width-x_min, card_height-y_min], [card_width-x_max, card_height-y_min]], dtype=np.float32)
ref_corners = np.array([ref_corner_1, ref_corner_2])
img_resized = cv2.resize(cv2.imread(img_file, cv2.IMREAD_UNCHANGED), (card_width, card_height))
show_image(img_resized, polygons=[ref_corner_1, ref_corner_2])

# Finding the convex hulls
This function 'extract_hull' finds the convex hull in one of the corner of a card image.
The convex hull is the minimal convex polygon that contains both the value and the suit symbols. 

In [ ]:
def extract_hull(image, region, debug="no"):
    kernel = np.ones((3,3),np.uint8)
    region = region.astype(int)
    x1 = int(region[0][0])
    y1 = int(region[0][1])
    x2 = int(region[2][0])
    y2 = int(region[2][1])
    w = x2 - x1
    h = y2 - y1
    zone = image[y1:y2, x1:x2].copy()
    temp_cnt = np.zeros_like(zone)
    gray = cv2.cvtColor(zone, cv2.COLOR_BGR2GRAY)
    thld = cv2.Canny(gray, 30, 200)
    thld = cv2.dilate(thld, kernel, iterations=1)
    if debug != "no": cv2.imshow("thld", thld)
    contours, _ = cv2.findContours(thld.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    min_area = 30
    min_solidity = 0.35
    concat_contour = None
    hull_in_img = None
    ok = True
    for c in contours:
        area = cv2.contourArea(c)
        hull = cv2.convexHull(c)
        hull_area = cv2.contourArea(hull)
        solidity = float(area) / hull_area
        M = cv2.moments(c)
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        if area >= min_area and abs(w/2-cx) < w*0.2 and abs(h/2-cy) < h*0.4 and solidity > min_solidity:
            if debug != "no":
                cv2.drawContours(zone, [c], 0, (255,0,0), -1)
            if concat_contour is None:
                concat_contour = c
            else:
                concat_contour = np.concatenate((concat_contour, c))
        if debug != "no" and solidity <= min_solidity:
            print("Solidity", solidity)
            cv2.drawContours(temp_cnt, [c], 0, 255, 2)
            cv2.imshow("Strange contours", temp_cnt)
    if concat_contour is not None:
        hull = cv2.convexHull(concat_contour)
        hull_area = cv2.contourArea(hull)
        min_hull_area = 940
        max_hull_area = 2120
        if hull_area < min_hull_area or hull_area > max_hull_area:
            ok = False
            if debug != "no":
                print("Hull area=", hull_area, "too large or too small")
        hull_in_img = hull + region[0]
    else:
        ok = False
    if debug != "no":
        if concat_contour is not None:
            cv2.drawContours(zone, [hull], 0, (0,255,0), 1)
            cv2.drawContours(image, [hull_in_img], 0, (0,255,0), 1)
        cv2.imshow("Zone", zone)
        cv2.imshow("Image", image)
        if ok and debug != "pause_always":
            key = cv2.waitKey(1)
        else:
            key = cv2.waitKey(0)
        if key == 27:
            return None
    return hull_in_img

In [ ]:
# Test find_hull on a random card image
# Run a few times...
imgs_fns = glob(card_img_dir + "/*.png")
img_fn = random.choice(imgs_fns)
# Get the corners of the card
cornerXmin, cornerXmax, cornerYmin, cornerYmax = extract_corners(img_fn.split("/")[-1])
# Get the reference corners of the card
refCornerHL = np.array([[cornerXmin,cornerYmin],[cornerXmax,cornerYmin],[cornerXmax,cornerYmax],[cornerXmin,cornerYmax]],dtype=np.float32)
refCornerLR = np.array([[card_width-cornerXmax,card_height-cornerYmax],[card_width-cornerXmin,card_height-cornerYmax],
                      [card_width-cornerXmin,card_height-cornerYmin],[card_width-cornerXmax,card_height-cornerYmin]],dtype=np.float32)
refCorners = np.array([refCornerHL,refCornerLR])

# Read and resize the image
img = cv2.resize(cv2.imread(img_fn, cv2.IMREAD_UNCHANGED), (card_width, card_height))

hullHL = extract_hull(img,refCornerHL)
if hullHL is None:
    print(f"No hull found in {img_fn}")
hullLR = extract_hull(img,refCornerLR)
show_image(img,[refCornerHL,refCornerLR,hullHL,hullLR])

#### Card Images Augmentation
This is a one time procedure, then we will directly use the pickle file. Create a folder for each card with augmented images of the original card (augmentation = blur/light alteration)

In [ ]:
# aug_dir = "data_classification/cards_aug"
# img_fns = glob(card_img_dir + "/*.png")
# img_fns = [os.path.basename(img_fn) for img_fn in img_fns]
# img_fns = [img_fn.split(".")[0] for img_fn in img_fns]
# img_fns = list(set(img_fns)) # remove duplicates
# img_fns = sorted(img_fns) # sort the list of image names
# # Create a folder for each image name
# for img_fn in img_fns:
#     folder_name = os.path.join(aug_dir, img_fn)
#     if not os.path.exists(folder_name):
#         os.makedirs(folder_name)
#     # Create 50 images in the folder with random transformations
#     for i in range(50):
#         #pick the image
#         img = select_image(card_img_dir, img_fn + ".png")
#         # resize the image to the size of the card
#         img = cv2.resize(img, (card_width, card_height))
#         # apply random transformations to the image
#         # stronger blur
#         if random.random() > 0.5:  # Randomly apply blur
#             ksize = random.choice([3])  # Only use a smaller kernel size
#             img = cv2.GaussianBlur(img, (ksize, ksize), 0)

#         # moderate lightning adjustment
#         hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#         h, s, v = cv2.split(hsv)
#         v = cv2.add(v, random.randint(-25, 25))  # Moderate brightness adjustment
#         s = cv2.add(s, random.randint(-15, 15))  # Moderate saturation adjustment
#         hsv = cv2.merge((h, s, v))
#         img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

#         # moderate color alteration
#         alpha = random.uniform(0.85, 1.15)  # Moderate weight for color shift
#         beta = random.randint(-10, 10)      # Moderate bias for color shift
#         img = cv2.addWeighted(img, alpha, img, 0, beta)
#         # save the image in the folder with the name of the image + "_" + str(i) + ".png"
#         img_name = os.path.join(folder_name, img_fn + "_" + str(i) + ".png")
#         cv2.imwrite(img_name, img)

### Load all card image, calculate their convex hulls and save the whole in a pickle file

This is a one time procedure as well. 
The structure saved in the pickle file is a dictionnary named 'cards' of lists of triplets (img,hullHL,hullLR). 

In [ ]:

# cards={}
# for suit in suit_types:
#     for value in value_types:
#         card_name = value+suit        
#         card_dir = os.path.join(aug_dir,card_name)
#         if not os.path.isdir(card_dir):
#             print(f"!!! {card_dir} does not exist !!!")
#             continue
#         cards[card_name] = []
#         for f in glob(card_dir+"/*.png"):
#             img = cv2.imread(f,cv2.IMREAD_UNCHANGED)
#             # Get the corners of the card
#             cornerXmin, cornerXmax, cornerYmin, cornerYmax = extract_corners(f.split("/")[-1])
#             # Get the reference corners of the card
#             refCornerHL = np.array([[cornerXmin,cornerYmin],[cornerXmax,cornerYmin],
#                                   [cornerXmax,cornerYmax],[cornerXmin,cornerYmax]],dtype=np.float32)
#             refCornerLR = np.array([[card_width-cornerXmax,card_height-cornerYmax],[card_width-cornerXmin,card_height-cornerYmax],
#                                   [card_width-cornerXmin,card_height-cornerYmin],[card_width-cornerXmax,card_height-cornerYmin]],dtype=np.float32)
#             refCorners = np.array([refCornerHL,refCornerLR])
#             hullHL = extract_hull(img,refCornerHL)
#             if hullHL is None: 
#                 print(f"File {f} not used.")
#                 continue
#             hullLR = extract_hull(img,refCornerLR)
#             if hullLR is None: 
#                 print(f"File {f} not used.")
#                 continue
#             # We store the image in "rgb" format (we don't need opencv anymore)
#             img = cv2.cvtColor(img,cv2.COLOR_BGRA2RGBA)
#             cards[card_name].append((img,hullHL,hullLR))
#         # print(f"Nb images for {card_name} : {len(cards[card_name])}")



# print("Saved in :",pickle_file)
# # Ensure the directory for the pickle file exists
# path = "data_classification"
# pickle.dump(cards, open(pickle_file, 'wb'))

# cv2.destroyAllWindows()

### Load the cards pickle file in 'cards'

In [ ]:
class Cards():
    def __init__(self,cards_pck_fn=pickle_file):
        self._cards=pickle.load(open(cards_pck_fn,'rb'))
        # self._cards is a dictionary where keys are card names (ex:'Kc') and values are lists of (img,hullHL,hullLR) 
        self._nb_cards_by_value={k:len(self._cards[k]) for k in self._cards}
        print("Nb of cards loaded per name :", self._nb_cards_by_value)
        
    def get_random(self, card_name=None, display=False):
        if card_name is None:
            card_name= random.choice(list(self._cards.keys()))
        card,hull1,hull2=self._cards[card_name][random.randint(0,self._nb_cards_by_value[card_name]-1)]
        if display:
            if display: show_image(card,[hull1,hull2],"rgb")
        return card,card_name,hull1,hull2
    
cards = Cards()

In [ ]:
# Test: display a random card
_ = cards.get_random(display=True)

# Generating a scene
We can now generate scenes. We are considering here only 2 kinds of scene:
1. a scene with 2 cards: each card is randomly transformed (scaled, rotated, translated) independantly from the other;
2. a scene with 3 cards : the 3 cards are grouped together (a bit randomly) to form a fan, then the group is randomly transformed.


In [ ]:
class Backgrounds():
    def __init__(self,backgrounds_dir=backgrounds_folder):
        self._images = []
        for img in glob(backgrounds_dir+"/*.jpg"):
            img = mpimg.imread(img)
            # We convert the image to RGBA format
            if img.shape[2]==3:
                img = cv2.cvtColor(img,cv2.COLOR_BGR2RGBA)
            else:
                img = cv2.cvtColor(img,cv2.COLOR_BGRA2RGBA)
            self._images.append(img)
        self._nb_images=len(self._images)
        print("Nb of images loaded :", self._nb_images)
    def get_random(self, display=False):
        bg = self._images[random.randint(0,self._nb_images-1)]
        if display: plt.imshow(bg)
        return bg[:,:,:3] # We remove the alpha channel if it exists
    
backgrounds = Backgrounds()

In [ ]:
# Test: display a random background
bg = backgrounds.get_random(display=True)


### To save bounding boxes annotations in Pascal VOC format 
http://host.robots.ox.ac.uk/pascal/VOC/voc2008/htmldoc/

In [ ]:
xml_body_1 = """<annotation>
        <folder>FOLDER</folder>
        <filename>{FILENAME}</filename>
        <path>{PATH}</path>
        <source>
                <database>Unknown</database>
        </source>
        <size>
                <width>{WIDTH}</width>
                <height>{HEIGHT}</height>
                <depth>3</depth>
        </size>
"""
xml_object = """ <object>
                <name>{CLASS}</name>
                <pose>Unspecified</pose>
                <truncated>0</truncated>
                <difficult>0</difficult>
                <bndbox>
                        <xmin>{XMIN}</xmin>
                        <ymin>{YMIN}</ymin>
                        <xmax>{XMAX}</xmax>
                        <ymax>{YMAX}</ymax>
                </bndbox>
        </object>
"""
xml_body_2 = """</annotation>        
"""

def create_voc_xml(xml_file, img_file,listbba,display=False):
    with open(xml_file,"w") as f:
        f.write(xml_body_1.format(**{'FILENAME':os.path.basename(img_file), 'PATH':img_file,'WIDTH':imgW,'HEIGHT':imgH}))
        for bba in listbba:            
            f.write(xml_object.format(**{'CLASS':bba.classname,'XMIN':bba.x1,'YMIN':bba.y1,'XMAX':bba.x2,'YMAX':bba.y2}))
        f.write(xml_body_2)
        if display: print("New xml",xml_file)
        


In [ ]:
# Scenario with 2 cards:
# The original image of a card has the shape (cardH,cardW,4)
# We first paste it in a zero image of shape (imgH,imgW,4) at position decalX, decalY
# so that the original image is centerd in the zero image
decalX = int((imgW-card_width)/2)
decalY = int((imgH-card_height)/2)

# Scenario with 3 cards : decal values are different
decalX3 = int((imgW-card_width)/2)
decalY3 = int((imgH-card_height)/2)

def kps_to_polygon(kps):
    """
        Convert imgaug keypoints to shapely polygon
    """
    pts = [(kp.x,kp.y) for kp in kps]
    return Polygon(pts)

def hull_to_kps(hull, decalX=decalX, decalY=decalY):
    """
        Convert hull to imgaug keypoints
    """
    # hull is a cv2.Contour, shape : Nx1x2
    kps = [ia.Keypoint(x=p[0]+decalX,y=p[1]+decalY) for p in hull.reshape(-1,2)]
    kps = ia.KeypointsOnImage(kps, shape=(imgH,imgW,3))
    return kps

def kps_to_BB(kps):
    """
        Determine imgaug bounding box from imgaug keypoints
    """
    extend = 3 # To make the bounding box a little bit bigger
    kpsx = [kp.x for kp in kps.keypoints]
    minx = max(0,int(min(kpsx)-extend))
    maxx = min(imgW,int(max(kpsx)+extend))
    kpsy = [kp.y for kp in kps.keypoints]
    miny = max(0,int(min(kpsy)-extend))
    maxy = min(imgH,int(max(kpsy)+extend))
    if minx == maxx or miny == maxy:
        return None
    else:
        return ia.BoundingBox(x1 = minx,y1 = miny,x2 = maxx,y2 = maxy)


# imgaug keypoints of the bounding box of a whole card
cardKP = ia.KeypointsOnImage([
    ia.Keypoint(x = decalX,y = decalY),
    ia.Keypoint(x = decalX+card_width,y = decalY),   
    ia.Keypoint(x = decalX+card_width,y = decalY+card_height),
    ia.Keypoint(x = decalX,y = decalY+card_height)
    ], shape = (imgH,imgW,3))

# imgaug transformation for one card in scenario with 2 cards
transform_1card = iaa.Sequential([
    iaa.Affine(scale = [0.65,1]),
    iaa.Affine(rotate = (-180,160)),
    iaa.Affine(translate_percent = {"x":(-0.25,0.25),"y":(-0.25,0.25)}),
])

# For the 3 cards scenario, we use 3 imgaug transforms, the first 2 are for individual cards, 
# and the third one for the group of 3 cards
trans_rot1 = iaa.Sequential([
    iaa.Affine(translate_px = {"x": (10, 20)}),
    iaa.Affine(rotate = (27,35))
])
trans_rot2 = iaa.Sequential([
    iaa.Affine(translate_px = {"x": (0, 5)}),
    iaa.Affine(rotate = (20,25))
])
transform_3cards = iaa.Sequential([
    iaa.Affine(translate_px = {"x":decalX-decalX3,"y":decalY-decalY3}),
    iaa.Affine(scale = [0.65,1]),
    iaa.Affine(rotate = (-180,180)),
    iaa.Affine(translate_percent = {"x":(-0.2,0.2),"y":(-0.2,0.2)})   
])

# imgaug transformation for the background
scaleBg = iaa.Scale({"height": imgH, "width": imgW})

def augment(img, list_kps, seq, restart=True):
    """
        Apply augmentation 'seq' to image 'img' and keypoints 'list_kps'
    """ 
    while True:
        if restart:
            myseq = seq.to_deterministic()
        else:
            myseq = seq
        # Augment image, keypoints and bbs 
        img_aug = myseq.augment_images([img])[0]
        list_kps_aug = [myseq.augment_keypoints([kp])[0] for kp in list_kps]
        list_bbs = [kps_to_BB(list_kps_aug[1]),kps_to_BB(list_kps_aug[2])]
        valid = True
        # Check the card bounding box stays inside the image
        for bb in list_bbs:
            if bb is None or int(round(bb.x2)) >= imgW or int(round(bb.y2)) >= imgH or int(bb.x1)<=0 or int(bb.y1)<=0:
                valid=False
                break
        if valid: break
        elif not restart:
            img_aug=None
            break
                
    return img_aug,list_kps_aug,list_bbs

class BBA:  # Bounding box + annotations
    def __init__(self,bb,classname):      
        self.x1=int(round(bb.x1))
        self.y1=int(round(bb.y1))
        self.x2=int(round(bb.x2))
        self.y2=int(round(bb.y2))
        self.classname=classname

class Scene:
    def __init__(self,bg,img1, class1, hulla1,hullb1,img2, class2,hulla2,hullb2,img3=None, class3=None,hulla3=None,hullb3=None):
        if img3 is not None:
            self.create3CardsScene(bg,img1, class1, hulla1,hullb1,img2, class2,hulla2,hullb2,img3, class3,hulla3,hullb3)
        else:
            self.create2CardsScene(bg,img1, class1, hulla1,hullb1,img2, class2,hulla2,hullb2)

    def create2CardsScene(self,bg,img1, class1, hulla1,hullb1,img2, class2,hulla2,hullb2):
        kpsa1 = hull_to_kps(hulla1)
        kpsb1 = hull_to_kps(hullb1)
        kpsa2 = hull_to_kps(hulla2)
        kpsb2 = hull_to_kps(hullb2)
        
        # Randomly transform 1st card
        self.img1 = np.zeros((imgH,imgW,4),dtype=np.uint8)
        self.img1[decalY:decalY+card_height,decalX:decalX+card_width,:] = img1
        self.img1,self.lkps1,self.bbs1=augment(self.img1,[cardKP,kpsa1,kpsb1],transform_1card)

        # Randomly transform 2nd card. We want that card 2 does not partially cover a corner of 1 card.
        # If so, we apply a new random transform to card 2
        while True:
            self.listbba = []
            self.img2 = np.zeros((imgH,imgW,4),dtype=np.uint8)
            self.img2[decalY:decalY+card_height,decalX:decalX+card_width,:] = img2
            self.img2,self.lkps2,self.bbs2 = augment(self.img2,[cardKP,kpsa2,kpsb2],transform_1card)

            # mainPoly2: shapely polygon of card 2
            mainPoly2 = kps_to_polygon(self.lkps2[0].keypoints[0:4])
            invalid = False
            intersect_ratio = 0.1
            for i in range(1,3):
                # smallPoly1: shapely polygon of one of the hull of card 1
                smallPoly1 = kps_to_polygon(self.lkps1[i].keypoints[:])
                a = smallPoly1.area
                # We calculate area of the intersection of card 1 corner with card 2
                intersect = mainPoly2.intersection(smallPoly1)
                ai = intersect.area
                # If intersection area is small enough, we accept card 2
                if (a-ai)/a > 1-intersect_ratio:
                    self.listbba.append(BBA(self.bbs1[i-1],class1))
                # If intersectio area is not small, but also not big enough, we want apply new transform to card 2
                elif (a-ai)/a>intersect_ratio:
                    invalid=True
                    break
                    
            if not invalid: break
            
        self.class1 = class1
        self.class2 = class2
        for bb in self.bbs2:
            self.listbba.append(BBA(bb,class2))
        # Construct final image of the scene by superimposing: bg, img1 and img2
        self.bg = scaleBg.augment_image(bg)

        alpha = self.img1[:, :, 3] / 255.0  # Normalize alpha to [0, 1]
        alpha = np.expand_dims(alpha, axis=-1)
        self.final = (self.img1[:, :, :3] * alpha + self.bg * (1 - alpha)).astype(np.uint8)

        alpha2 = self.img2[:, :, 3] / 255.0  # Normalize alpha to [0, 1]
        alpha2 = np.expand_dims(alpha2, axis=-1)
        self.final = (self.img2[:, :, :3] * alpha2 + self.final * (1 - alpha2)).astype(np.uint8)
        
              
    def create3CardsScene(self,bg,img1, class1, hulla1,hullb1,img2, class2,hulla2,hullb2,img3, class3,hulla3,hullb3):
        
        kpsa1 = hull_to_kps(hulla1,decalX3,decalY3)
        kpsb1 = hull_to_kps(hullb1,decalX3,decalY3)
        kpsa2 = hull_to_kps(hulla2,decalX3,decalY3)
        kpsb2 = hull_to_kps(hullb2,decalX3,decalY3)
        kpsa3 = hull_to_kps(hulla3,decalX3,decalY3)
        kpsb3 = hull_to_kps(hullb3,decalX3,decalY3)
        
        # Create empty images for the cards
        self.img3 = np.zeros((imgH, imgW, 4), dtype=np.uint8)
        # Calculate valid placement coordinates
        y_start = max(0, decalY3)
        y_end = min(imgH, decalY3 + card_height)
        x_start = max(0, decalX3)
        x_end = min(imgW, decalX3 + card_width)
        
        # Calculate source image region to copy from
        src_y_start = max(0, -decalY3)
        src_y_end = src_y_start + (y_end - y_start)
        src_x_start = max(0, -decalX3)
        src_x_end = src_x_start + (x_end - x_start)
        
        # Place card 3 safely
        self.img3[y_start:y_end, x_start:x_end, :] = img3[src_y_start:src_y_end, src_x_start:src_x_end, :]
        self.img3, self.lkps3, self.bbs3 = augment(self.img3, [cardKP, kpsa3, kpsb3], trans_rot1)
        
        # Same for card 2
        self.img2 = np.zeros((imgH, imgW, 4), dtype=np.uint8)
        self.img2[y_start:y_end, x_start:x_end, :] = img2[src_y_start:src_y_end, src_x_start:src_x_end, :]
        self.img2, self.lkps2, self.bbs2 = augment(self.img2, [cardKP, kpsa2, kpsb2], trans_rot2)
        
        # Same for card 1
        self.img1 = np.zeros((imgH, imgW, 4), dtype=np.uint8)
        self.img1[y_start:y_end, x_start:x_end, :] = img1[src_y_start:src_y_end, src_x_start:src_x_end, :]
        
        # Apply transformations to the group of cards
        while True:
            det_transform_3cards = transform_3cards.to_deterministic()
            _img3, _lkps3, self.bbs3 = augment(self.img3, self.lkps3, det_transform_3cards, False)
            if _img3 is None: continue
            _img2, _lkps2, self.bbs2 = augment(self.img2, self.lkps2, det_transform_3cards, False)
            if _img2 is None: continue
            _img1, self.lkps1, self.bbs1 = augment(self.img1, [cardKP, kpsa1, kpsb1], det_transform_3cards, False)
            if _img1 is None: continue
            break
        
        self.img3 = _img3
        self.lkps3 = _lkps3
        self.img2 = _img2
        self.lkps2 = _lkps2
        self.img1 = _img1
        
        self.class1 = class1
        self.class2 = class2
        self.class3 = class3
        self.listbba = [BBA(self.bbs1[0], class1), BBA(self.bbs2[0], class2), BBA(self.bbs3[0], class3), BBA(self.bbs3[1], class3)]
        
        # Construct final image of the scene by superimposing: bg, img1, img2 and img3
        self.bg = scaleBg.augment_image(bg)

        alpha = self.img1[:, :, 3] / 255.0  # Normalize alpha to [0, 1]
        alpha = np.expand_dims(alpha, axis=-1)
        self.final = (self.img1[:, :, :3] * alpha + self.bg * (1 - alpha)).astype(np.uint8)

        alpha2 = self.img2[:, :, 3] / 255.0  # Normalize alpha to [0, 1]
        alpha2 = np.expand_dims(alpha2, axis=-1)
        self.final = (self.img2[:, :, :3] * alpha2 + self.final * (1 - alpha2)).astype(np.uint8)

        alpha3 = self.img3[:, :, 3] / 255.0  # Normalize alpha to [0, 1]
        alpha3 = np.expand_dims(alpha3, axis=-1)
        self.final = (self.img3[:, :, :3] * alpha3 + self.final * (1 - alpha3)).astype(np.uint8)

    def display(self):
        fig,ax = plt.subplots(1,figsize=(8,8))
        ax.imshow(self.final)
        for bb in self.listbba:
            rect = patches.Rectangle((bb.x1,bb.y1),bb.x2-bb.x1,bb.y2-bb.y1,linewidth=1,edgecolor='b',facecolor='none')
            ax.add_patch(rect)
    def res(self):
        return self.final
    def write_files(self,save_dir,idx,display=False):
        jpg_fn, xml_fn = unique_filename(save_dir, ["jpg","xml"], idx)
        plt.imsave(jpg_fn,self.final)
        if display: print("New image saved in",jpg_fn)
        create_voc_xml(xml_fn,jpg_fn, self.listbba,display=display)


In [ ]:
# Test generation of a scene with 2 cards
bg = backgrounds.get_random()
print("Background shape",bg.shape)
img1,card_val1,hulla1,hullb1 = cards.get_random()
img2,card_val2,hulla2,hullb2 = cards.get_random()

newimg = Scene(bg,img1,card_val1,hulla1,hullb1,img2,card_val2,hulla2,hullb2)
print("Final image shape",newimg.res().shape)
newimg.display()

In [ ]:
# Test generation of a scene with 3 cards
bg = backgrounds.get_random()
img1,card_val1,hulla1,hullb1 = cards.get_random()
img2,card_val2,hulla2,hullb2 = cards.get_random()
img3,card_val3,hulla3,hullb3 = cards.get_random()

newimg = Scene(bg,img1,card_val1,hulla1,hullb1,img2,card_val2,hulla2,hullb2,img3,card_val3,hulla3,hullb3)
newimg.display()

## Generate the datasets


In [ ]:
nb_cards_to_generate = 14000
save_dir = os.path.join(os.getcwd(),"data_yolo/train/")

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

for i in tqdm(range(nb_cards_to_generate)):
    bg=backgrounds.get_random()
    img1,card_val1,hulla1,hullb1=cards.get_random()
    img2,card_val2,hulla2,hullb2=cards.get_random()
    if random.random() > 0.5:
        img3,card_val3,hulla3,hullb3=cards.get_random()
        newimg=Scene(bg,img1,card_val1,hulla1,hullb1,img2,card_val2,hulla2,hullb2,img3,card_val3,hulla3,hullb3)
    else:
        newimg=Scene(bg,img1,card_val1,hulla1,hullb1,img2,card_val2,hulla2,hullb2)
    
    newimg.write_files(save_dir, i)

In [ ]:
nb_cards_to_generate=4000
save_dir = os.path.join(os.getcwd(),"data_yolo/development/")

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

for i in tqdm(range(nb_cards_to_generate)):
    bg=backgrounds.get_random()
    img1,card_val1,hulla1,hullb1=cards.get_random()
    img2,card_val2,hulla2,hullb2=cards.get_random()
    if random.random() > 0.5:
        img3,card_val3,hulla3,hullb3=cards.get_random()
        newimg=Scene(bg,img1,card_val1,hulla1,hullb1,img2,card_val2,hulla2,hullb2,img3,card_val3,hulla3,hullb3)
    else:
        newimg=Scene(bg,img1,card_val1,hulla1,hullb1,img2,card_val2,hulla2,hullb2)
    
    newimg.write_files(save_dir, i)


In [ ]:
nb_cards_to_generate=2000
save_dir = os.path.join(os.getcwd(),"data_yolo/test/")

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

for i in tqdm(range(nb_cards_to_generate)):
    bg=backgrounds.get_random()
    img1,card_val1,hulla1,hullb1=cards.get_random()
    img2,card_val2,hulla2,hullb2=cards.get_random()
    if random.random() > 0.5:
        img3,card_val3,hulla3,hullb3=cards.get_random()
        newimg=Scene(bg,img1,card_val1,hulla1,hullb1,img2,card_val2,hulla2,hullb2,img3,card_val3,hulla3,hullb3)
    else:
        newimg=Scene(bg,img1,card_val1,hulla1,hullb1,img2,card_val2,hulla2,hullb2)
    
    newimg.write_files(save_dir, i)


### Conversion from Pascal VOC to txt

In [ ]:
!python utils_data/convert_voc_yolo.py data_yolo/train/annotations utils_data/cards.names data_yolo/train/labels

In [ ]:
!python utils_data/convert_voc_yolo.py data_yolo/development/annotations utils_data/cards.names data_yolo/development/labels

In [ ]:
!python utils_data/convert_voc_yolo.py data_yolo/test/annotations utils_data/cards.names data_yolo/test/labels